In [2]:
#https://github.com/gboeing/osmnx-examples/blob/master/notebooks/13-isolines-isochrones.ipynb
import geopandas as gpd
import pandas as pd
import folium
import shapely
import numpy as np
#import pyproj
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import cascaded_union

## Putting all polys into 1 file

In [4]:
x = np.arange(1,73)

all_polys = gpd.read_file('gis_data/isochrones/ER_1.json')

for i in x:
    i = i + 1
    one_poly_file = "gis_data/isochrones/ER_" + str(i) + ".json"

    poly_add = gpd.read_file(one_poly_file)

    all_polys = pd.concat([all_polys,poly_add])
    
all_polys = all_polys.sort_values(by=['drivetime'], ascending=False)

all_polys.to_file("gis_data/isochrones/all_ER_polys.json", driver="GeoJSON")

print(73*5)
print(len(all_polys))

365
365


In [7]:
all_polys = gpd.read_file('gis_data/isochrones/all_ER_polys.json')

## this creates a 400KB file, which is really difficult to use
m = folium.Map(location=[40.353944, -74.063478])

folium.GeoJson(
    all_polys,
    name='geojson',    
    
    ## controls the fill of the geo regions; applying colorscale based on variable
    style_function=lambda x: {"weight":1
                ## color of outline
                , 'color': '#545453'
                ## pulling fill color from the geopandas file
                , 'fillColor': x['properties']['color']
                , 'fillOpacity': 0.65 },
    
                ## changes styling of geo regions upon hover
                highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
               
                ## tooltip can include information from any column in the GeoPandas dataframe   
                tooltip=folium.features.GeoJsonTooltip(
                fields=['drivetime'],
                aliases=['Drivetime (Mins):'])
    
).add_to(m)

m.save('nj_drivetime_all.html')

## Let's examine an individual polygon to learn more about it

In [6]:
one_poly = gpd.read_file('gis_data/isochrones/ER_1.json')

In [19]:
## each polygon has 
print(len(one_poly['geometry'])) ## 5 geometries (5,10,15,20,25)

geom = one_poly['geometry'][0] 

print(len(geom.exterior.coords)) ## first geom has 32K+ coordinates

x, y = geom.exterior.coords.xy ## coordinates are in Mercator format, with 10 digits

print(x[0])

5
32046
532269.5901560324


In [33]:
#https://shapely.readthedocs.io/en/stable/manual.html#object.simplify

simple_poly = one_poly
simple_poly['geometry'] = simple_poly['geometry'].simplify(2, preserve_topology = False)

geom = simple_poly['geometry'][0] 

print(len(geom.exterior.coords)) ## first geom has 6K+ coordinates

x, y = geom.exterior.coords.xy ## coordinates are in Mercator format, with 10 digits

print(x[0])

simple_poly.to_file("gis_data/isochrones/simple/ER_1_r.json", driver="GeoJSON") ## polygon is 20% of the size

5986
532269.5901560324


## Applying the reduction to all of the drivetime polygons

In [39]:
x = np.arange(1,74)

for i in x:
    one_poly_file = "gis_data/isochrones/ER_" + str(i) + ".json"
    one_poly = gpd.read_file(one_poly_file)
    one_poly['geometry'] = one_poly['geometry'].simplify(2, preserve_topology = False)
    
    one_poly_file = "gis_data/isochrones/simple/ER_" + str(i) + "_r.json"
    
    one_poly.to_file(one_poly_file, driver="GeoJSON") 

In [40]:
## re-running poly consolidation on the simple polys
x = np.arange(1,73)

all_polys = gpd.read_file('gis_data/isochrones/simple/ER_1_r.json')

for i in x:
    i = i + 1
    one_poly_file = "gis_data/isochrones/simple/ER_" + str(i) + "_r.json"

    poly_add = gpd.read_file(one_poly_file)

    all_polys = pd.concat([all_polys,poly_add])
    
all_polys = all_polys.sort_values(by=['drivetime'], ascending=False)

all_polys.to_file("gis_data/isochrones/simple/all_ER_polys_r.json", driver="GeoJSON")

print(73*5)
print(len(all_polys))

365
365


In [15]:
all_polys = gpd.read_file('gis_data/isochrones/simple/all_ER_polys_r.json')
all_polys['geometry'] = all_polys['geometry'].simplify(2.5, preserve_topology = False)

In [17]:
all_polys = gpd.read_file('gis_data/isochrones/simple/all_ER_polys_r.json')

## this creates a 400KB file, which is really difficult to use
m = folium.Map(location=[40.353944, -74.063478])

folium.GeoJson(
    all_polys,
    name='geojson',    
    
    ## controls the fill of the geo regions; applying colorscale based on variable
    style_function=lambda x: {"weight":1
                ## color of outline
                , 'color': '#545453'
                ## pulling fill color from the geopandas file
                , 'fillColor': x['properties']['color']
                , 'fillOpacity': 0.65 },
    
                ## changes styling of geo regions upon hover
                highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
               
                ## tooltip can include information from any column in the GeoPandas dataframe   
                tooltip=folium.features.GeoJsonTooltip(
                fields=['ER','drivetime'],
                aliases=['Facility', 'Drivetime (Mins):'])
    
).add_to(m)

m.save('nj_drivetime_all_simple.html')

## Unioning the polygons so we have 1 multi polygon per drivetime

In [7]:
all_polys = gpd.read_file('gis_data/isochrones/simple/all_ER_polys_r.json')

trip_times = [5, 10, 15, 20, 25] #in minutes

for time in trip_times:

    poly_sub = all_polys[all_polys['drivetime'] == time]
    poly_sub = poly_sub.sort_values(by=['ER'])
    
    polygon1 = poly_sub.iloc[0,3]
    polygon2 = poly_sub.iloc[1,3]
    
    polygons = [polygon1, polygon2]
    
    u = cascaded_union(polygons)

    color = poly_sub.iloc[0,0]  
    drivetime = time
    ER = "ER_U"
    
    poly_union = gpd.GeoDataFrame([[color,drivetime, ER, u]], columns = ['color', 'drivetime','ER', 'geometry'], crs="EPSG:26918")
    
    print(poly_union)
    
    x = np.arange(2,73)

    for i in x:
    
        polygon1 = poly_union.iloc[0,3]
        polygon2 = poly_sub.iloc[i,3]

        polygons = [polygon1, polygon2]

        u = cascaded_union(polygons)
    
        poly_union = gpd.GeoDataFrame([[color,drivetime, ER, u]], columns = ['color', 'drivetime','ER', 'geometry'], crs="EPSG:26918")
        
        filename = "gis_data/isochrones/simple/ER_" + str(time) + "_min_r.json"
        
        poly_union.to_file(filename, driver = "GeoJSON")

     color  drivetime    ER                                           geometry
0  #f0f921          5  ER_U  MULTIPOLYGON (((537465.107 4355729.742, 537485...
     color  drivetime    ER                                           geometry
0  #f89540         10  ER_U  MULTIPOLYGON (((535389.887 4352237.245, 535376...
     color  drivetime    ER                                           geometry
0  #cc4778         15  ER_U  MULTIPOLYGON (((536009.667 4348868.608, 536017...
     color  drivetime    ER                                           geometry
0  #7e03a8         20  ER_U  MULTIPOLYGON (((534866.661 4345886.093, 534873...
     color  drivetime    ER                                           geometry
0  #0d0887         25  ER_U  MULTIPOLYGON (((532269.590 4342730.366, 532279...


In [11]:
union_5 = gpd.read_file("gis_data/isochrones/simple/ER_5_min_r.json")
union_10 = gpd.read_file("gis_data/isochrones/simple/ER_10_min_r.json")
union_15 = gpd.read_file("gis_data/isochrones/simple/ER_15_min_r.json")
union_20 = gpd.read_file("gis_data/isochrones/simple/ER_20_min_r.json")
union_25 = gpd.read_file("gis_data/isochrones/simple/ER_25_min_r.json")

In [12]:
unions = pd.concat([union_5,union_10,union_15,union_20, union_25])
unions = unions.sort_values(by=['drivetime'], ascending=False)
unions.head()

,color,drivetime,ER,geometry
0,#0d0887,25,ER_U,"MULTIPOLYGON (((507532.432 4310443.621, 507511..."
0,#7e03a8,20,ER_U,"MULTIPOLYGON (((508488.824 4315084.509, 508480..."
0,#cc4778,15,ER_U,"MULTIPOLYGON (((511469.521 4317752.629, 511418..."
0,#f89540,10,ER_U,"MULTIPOLYGON (((513642.993 4321351.898, 513656..."
0,#f0f921,5,ER_U,"MULTIPOLYGON (((515362.915 4324919.423, 515372..."


In [14]:
## this creates a 400KB file, which is really difficult to use
m = folium.Map(location=[40.353944, -74.063478])

## controls the fill of the geo regions; applying colorscale based on variable
style = lambda x: {"weight":1
                ## color of outline
                , 'color': '#545453'
                ## pulling fill color from the geopandas file
                , 'fillColor': x['properties']['color']
                , 'fillOpacity': 0.65 }

drive_5 = folium.GeoJson(
    union_5,
    name='5 Minutes',    
    style_function= style,
    ## changes styling of geo regions upon hover
    highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
    #tooltip can include information from any column in the GeoPandas dataframe   
    tooltip=folium.features.GeoJsonTooltip(
    fields=['drivetime'],
    aliases=['Drivetime (Mins):'])
)

drive_10 = folium.GeoJson(
    union_10,
    name='10 Minutes',    
    style_function= style,
    ## changes styling of geo regions upon hover
    highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
    #tooltip can include information from any column in the GeoPandas dataframe   
    tooltip=folium.features.GeoJsonTooltip(
    fields=['drivetime'],
    aliases=['Drivetime (Mins):'])
)

drive_15 = folium.GeoJson(
    union_15,
    name='15 Minutes',    
    style_function= style,
    ## changes styling of geo regions upon hover
    highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
    #tooltip can include information from any column in the GeoPandas dataframe   
    tooltip=folium.features.GeoJsonTooltip(
    fields=['drivetime'],
    aliases=['Drivetime (Mins):'])
)

drive_20 = folium.GeoJson(
    union_20,
    name='20 Minutes',    
    style_function= style,
    ## changes styling of geo regions upon hover
    highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
    #tooltip can include information from any column in the GeoPandas dataframe   
    tooltip=folium.features.GeoJsonTooltip(
    fields=['drivetime'],
    aliases=['Drivetime (Mins):'])
)

drive_25 = folium.GeoJson(
    union_25,
    name='25 Minutes',    
    style_function= style,
    ## changes styling of geo regions upon hover
    highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
    #tooltip can include information from any column in the GeoPandas dataframe   
    tooltip=folium.features.GeoJsonTooltip(
    fields=['drivetime'],
    aliases=['Drivetime (Mins):'])
)

m.add_child(drive_25)
m.add_child(drive_20)
m.add_child(drive_15)
m.add_child(drive_10)
m.add_child(drive_5)

# add the layer control
folium.LayerControl().add_to(m)

m.save('nj_drivetime_union_simple.html')